ENVIRONMENT SETUP

In [11]:
import pandas as pd
import sqlite3
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("=" * 70)
print("ADVANCED SQL QUERIES")
print("=" * 70)

conn = sqlite3.connect('../data/processed/ecommerce.db')

def run_query(query, title):
    """Execute Query and Display Result"""
    print(f"\n{'=' * 70}")
    print(title)
    print('=' * 70)
    result = pd.read_sql_query(query, conn)
    print(result.to_string(index=False))
    return result

print("\nDatabase connection established")

ADVANCED SQL QUERIES

Database connection established


QUERY 11: RUNNING TOTAL REVENUE BY MONTH

In [34]:
# Running Total
query11 = """
SELECT
    order_year,
    order_month,
    ROUND(SUM(p.total_payment_value), 2) as monthly_revenue,
    ROUND(SUM(SUM(p.total_payment_value)) OVER (
        ORDER BY order_year, order_month
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    ), 2) as running_total_revenue
FROM orders o
JOIN order_payments p ON o.order_id = p.order_id
GROUP BY order_year, order_month
ORDER BY order_year, order_month
"""

result11 = run_query(query11, "QUERY 11: Running Total Revenue by Month")


QUERY 11: Running Total Revenue by Month
 order_year  order_month  monthly_revenue  running_total_revenue
       2016           10         46566.71               46566.71
       2016           12            19.62               46586.33
       2017            1        127545.67              174132.00
       2017            2        271298.65              445430.65
       2017            3        414369.39              859800.04
       2017            4        390952.18             1250752.22
       2017            5        567066.73             1817818.95
       2017            6        490225.60             2308044.55
       2017            7        566403.93             2874448.48
       2017            8        646000.61             3520449.09
       2017            9        701169.99             4221619.08
       2017           10        751140.27             4972759.35
       2017           11       1153528.05             6126287.40
       2017           12        843199.17       


QUERY 12: CUSTOMER LIFETIME VALUE SEGMENTATION (CTE)

In [35]:
query12 = """
WITH customer_ltv AS (
    SELECT
        c.customer_unique_id,
        c.customer_state,
        COUNT(DISTINCT o.order_id) as total_orders,
        ROUND(SUM(p.total_payment_value), 2) as lifetime_value,
        ROUND(AVG(p.total_payment_value), 2) as avg_order_value
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
    JOIN order_payments p ON o.order_id = p.order_id
    GROUP BY c.customer_unique_id, c.customer_state
),
ltv_segments AS(
    SELECT
        customer_unique_id,
        customer_state,
        total_orders,
        lifetime_value,
        avg_order_value,
        CASE
            WHEN lifetime_value >= 1000 THEN 'High Value'
            WHEN lifetime_value >= 500 THEN 'Medium Value'
            ELSE 'Low Value'
        END as value_segment
    FROM customer_ltv
)
SELECT
    value_segment,
    COUNT(*) as customer_count,
    ROUND(AVG(lifetime_value), 2) as avg_ltv,
    ROUND(AVG(total_orders), 2) as avg_orders,
    ROUND(SUM(lifetime_value), 2) as total_segment_revenue
FROM ltv_segments
GROUP BY value_segment
ORDER BY avg_ltv DESC
"""

result12 = run_query(query12, "QUERY 12: Customer Value Segmentation")


QUERY 12: Customer Value Segmentation
value_segment  customer_count  avg_ltv  avg_orders  total_segment_revenue
   High Value            1148  1584.78        1.11             1819322.15
 Medium Value            3116   679.49        1.13             2117295.97
    Low Value           89131   128.86        1.03            11485843.65


QUERY 13: MONTHLY CUSTOMER ACQUISITION COHORTS

In [36]:
query13 = """
WITH first_purchase AS (
    SELECT
        c.customer_unique_id,
        MIN(o.order_purchase_timestamp) as first_order_date,
        strftime('%Y=%m', MIN(o.order_purchase_timestamp)) as cohort_month
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
    GROUP BY c.customer_unique_id
)
SELECT
    cohort_month,
    COUNT(DISTINCT customer_unique_id) as customers_acquired,
    SUM(COUNT(DISTINCT customer_unique_id)) OVER (
        ORDER BY cohort_month
    ) as cumulative_customers
FROM first_purchase
GROUP BY cohort_month
ORDER BY cohort_month
"""

result13 = run_query(query13, "QUERY 13: Monthly Customer Acquisition Cohorts")


QUERY 13: Monthly Customer Acquisition Cohorts
cohort_month  customers_acquired  cumulative_customers
     2016=09                   1                     1
     2016=10                 262                   263
     2016=12                   1                   264
     2017=01                 717                   981
     2017=02                1628                  2609
     2017=03                2503                  5112
     2017=04                2256                  7368
     2017=05                3451                 10819
     2017=06                3037                 13856
     2017=07                3752                 17608
     2017=08                4057                 21665
     2017=09                4004                 25669
     2017=10                4328                 29997
     2017=11                7060                 37057
     2017=12                5338                 42395
     2018=01                6842                 49237
     2018=02     

QUERY 14: ORDER BASKET SIZE ANALYSIS

In [37]:
query14 = """
WITH order_size AS (
    SELECT
        order_id,
        COUNT(DISTINCT product_id) as items_count,
        ROUND(SUM(price), 2) as order_value
    FROM order_items
    GROUP BY order_id
)
SELECT
    CASE
        WHEN items_count = 1 THEN '1 item'
        WHEN items_count BETWEEN 2 AND 3 THEN '2-3 items'
        WHEN items_count BETWEEN 4 ANd 5 THEN '4-5 items'
        ELSE '6+ items'
    END as basket_size,
    COUNT(*) as order_count,
    ROUND(AVG(order_value), 2) as avg_order_value,
    ROUND(SUM(order_value), 2) as total_revenue,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) as pct_of_orders
FROM order_size
GROUP BY basket_size
ORDER BY
    CASE basket_size
        WHEN '1 item' THEN 1
        WHEN'2-3 items' THEN 2
        WHEN '4-5 items' THEN 3
        ELSE 4
    END
"""

result14 = run_query(query14, "QUERY 14: Order Basket Size Analysis")


QUERY 14: Order Basket Size Analysis
basket_size  order_count  avg_order_value  total_revenue  pct_of_orders
     1 item        95430           135.65    12944973.77          96.72
  2-3 items         3144           195.15      613551.12           3.19
  4-5 items           78           344.04       26835.13           0.08
   6+ items           14           448.83        6283.68           0.01


QUERY 15: STATE PERFORMANCE RANKINGS

In [38]:
query15 = """
SELECT
    c.customer_state,
    COUNT(DISTINCT o.order_id) as total_orders,
    ROUND(SUM(p.total_payment_value), 2) as total_revenue,
    ROUND(AVG(o.delivery_time_days), 1) as avg_delivery_days,
    RANK() OVER (ORDER BY SUM(p.total_payment_value) DESC) as revenue_rank,
    RANK() OVER (ORDER BY AVG(o.delivery_time_days) ASC) as delivery_rank
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
JOIN order_payments p ON o.order_id = p.order_id
WHERE o.delivery_time_days IS NOT NULL
GROUP BY c.customer_state
ORDER BY revenue_rank
"""

result15 = run_query(query15, "QUERY 15: State Performance Ranking")


QUERY 15: State Performance Ranking
customer_state  total_orders  total_revenue  avg_delivery_days  revenue_rank  delivery_rank
            SP         40493     5769081.27                8.3             1              1
            RJ         12350     2055690.45               14.8             2              7
            MG         11354     1819277.61               11.5             3              3
            RS          5344      861608.40               14.8             4              6
            PR          4923      781919.55               11.5             5              2
            SC          3546      595208.40               14.5             6              5
            BA          3256      591270.60               18.9             7             15
            DF          2080      346146.17               12.5             8              4
            GO          1957      334294.22               15.2             9              8
            ES          1995      317682.65

PART 2: SAVE QUERIES TO SQL FILE

In [39]:
# CELL 7: Export all queries to .sql file
queries_sql = """-- E-COMMERCE ANALYSIS - ADVANCED SQL QUERIES
-- Database: ecommerce.db
-- Date: January 6, 2026
-- Author: Christian Wedha

-- ============================================================
-- QUERY 11: Running Total Revenue by Month
-- Purpose: Calculate cumulative revenue growth over time
-- Technique: Window function (SUM OVER with frame specification)
-- ============================================================

{q11}

-- ============================================================
-- QUERY 12: Customer Value Segmentation
-- Purpose: Segment customers by lifetime value (High/Medium/Low)
-- Technique: Common Table Expressions (CTEs)
-- ============================================================

{q12}

-- ============================================================
-- QUERY 13: Monthly Customer Acquisition Cohorts
-- Purpose: Track new customer acquisition by month with cumulative total
-- Technique: CTE + window function for running sum
-- ============================================================

{q13}

-- ============================================================
-- QUERY 14: Order Basket Size Analysis
-- Purpose: Analyze distribution of items per order
-- Technique: Subquery + CASE WHEN categorization + percentage calculation
-- ============================================================

{q14}

-- ============================================================
-- QUERY 15: State Performance Rankings
-- Purpose: Rank states by revenue and delivery performance simultaneously
-- Technique: Multiple RANK() window functions
-- ============================================================

{q15}
""".format(
    q11=query11, 
    q12=query12, 
    q13=query13, 
    q14=query14, 
    q15=query15
)

# Save to file
with open('../sql/advanced_queries.sql', 'w') as f:
    f.write(queries_sql)

print("\n" + "=" * 70)
print("All queries saved to: sql/advanced_queries.sql")
print("=" * 70)


All queries saved to: sql/advanced_queries.sql


INSIGHT AND ANALYSIS

In [40]:
# CELL 8: Cross-query insights

print("\n" + "=" * 70)
print("BUSINESS INSIGHTS ACROSS ALL ADVANCED QUERIES")
print("=" * 70)

# Insight 1: Revenue growth rate analysis
print("\n" + "-" * 70)
print("INSIGHT 1: REVENUE GROWTH TRAJECTORY")
print("-" * 70)

# Calculate month-over-month growth from Query 11 results
result11['prev_month_revenue'] = result11['monthly_revenue'].shift(1)
result11['mom_growth_pct'] = (
    (result11['monthly_revenue'] - result11['prev_month_revenue']) / 
    result11['prev_month_revenue'] * 100
).round(2)

print("\nMonth-over-month growth rates:")
print(result11[['order_year', 'order_month', 'monthly_revenue', 'mom_growth_pct']].tail(12).to_string(index=False))

peak_growth_month = result11.loc[result11['mom_growth_pct'].idxmax()]
worst_growth_month = result11.loc[result11['mom_growth_pct'].idxmin()]

print(f"\nPeak growth month: {peak_growth_month['order_year']}-{peak_growth_month['order_month']:02.0f}")
print(f"  Growth: {peak_growth_month['mom_growth_pct']:.1f}%")
print(f"  Revenue: R$ {peak_growth_month['monthly_revenue']:,.2f}")

print(f"\nWorst growth month: {worst_growth_month['order_year']}-{worst_growth_month['order_month']:02.0f}")
print(f"  Growth: {worst_growth_month['mom_growth_pct']:.1f}%")
print(f"  Revenue: R$ {worst_growth_month['monthly_revenue']:,.2f}")

# Insight 2: Customer segment economics
print("\n" + "-" * 70)
print("INSIGHT 2: CUSTOMER SEGMENT ECONOMICS")
print("-" * 70)

total_customers = result12['customer_count'].sum()
total_revenue = result12['total_segment_revenue'].sum()

result12['pct_of_customers'] = (result12['customer_count'] / total_customers * 100).round(2)
result12['pct_of_revenue'] = (result12['total_segment_revenue'] / total_revenue * 100).round(2)

print("\nSegment distribution:")
print(result12[['value_segment', 'customer_count', 'pct_of_customers', 
                'total_segment_revenue', 'pct_of_revenue']].to_string(index=False))

high_value = result12[result12['value_segment'] == 'High Value'].iloc[0]
low_value = result12[result12['value_segment'] == 'Low Value'].iloc[0]

ltv_multiplier = high_value['avg_ltv'] / low_value['avg_ltv']

print(f"\nHigh-value customers are {ltv_multiplier:.1f}x more valuable than low-value")
print(f"But represent only {high_value['pct_of_customers']:.2f}% of customer base")
print(f"Yet contribute {high_value['pct_of_revenue']:.2f}% of total revenue")

# Insight 3: Acquisition vs retention
print("\n" + "-" * 70)
print("INSIGHT 3: ACQUISITION TRENDS")
print("-" * 70)

# Calculate acquisition growth rate
result13['prev_month_acq'] = result13['customers_acquired'].shift(1)
result13['acq_growth_pct'] = (
    (result13['customers_acquired'] - result13['prev_month_acq']) / 
    result13['prev_month_acq'] * 100
).round(2)

print("\nLast 12 months customer acquisition:")
print(result13[['cohort_month', 'customers_acquired', 'cumulative_customers', 'acq_growth_pct']].tail(12).to_string(index=False))

avg_monthly_acquisition = result13['customers_acquired'].mean()
peak_acquisition_month = result13.loc[result13['customers_acquired'].idxmax()]

print(f"\nAverage monthly acquisition: {avg_monthly_acquisition:.0f} customers")
print(f"Peak acquisition month: {peak_acquisition_month['cohort_month']}")
print(f"  Customers acquired: {peak_acquisition_month['customers_acquired']:.0f}")
print(f"  Cumulative total: {peak_acquisition_month['cumulative_customers']:.0f}")

# Insight 4: Basket behavior
print("\n" + "-" * 70)
print("INSIGHT 4: BASKET SIZE ECONOMICS")
print("-" * 70)

print("\nBasket size distribution:")
print(result14.to_string(index=False))

single_item_pct = result14[result14['basket_size'] == '1 item']['pct_of_orders'].values[0]
single_item_aov = result14[result14['basket_size'] == '1 item']['avg_order_value'].values[0]

multi_item = result14[result14['basket_size'] != '1 item']
multi_item_avg_aov = multi_item['avg_order_value'].mean()

aov_uplift = ((multi_item_avg_aov - single_item_aov) / single_item_aov * 100)

print(f"\n{single_item_pct:.1f}% of orders contain only 1 item")
print(f"Single-item AOV: R$ {single_item_aov:.2f}")
print(f"Multi-item AOV: R$ {multi_item_avg_aov:.2f}")
print(f"AOV uplift from cross-sell: {aov_uplift:.1f}%")

# Insight 5: State performance gaps
print("\n" + "-" * 70)
print("INSIGHT 5: STATE PERFORMANCE GAPS")
print("-" * 70)

# Find states with big ranking gaps
result15['rank_gap'] = abs(result15['revenue_rank'] - result15['delivery_rank'])
top_gap_states = result15.nlargest(5, 'rank_gap')

print("\nTop 5 states with largest performance gaps:")
print(top_gap_states[['customer_state', 'revenue_rank', 'delivery_rank', 'rank_gap']].to_string(index=False))

print("\nInterpretation:")
print("Large gaps indicate:")
print("  - High revenue + poor delivery = Fix logistics")
print("  - Low revenue + good delivery = Untapped potential")

# Summary statistics
print("\n" + "=" * 70)
print("DAY 6 COMPLETE - SUMMARY STATISTICS")
print("=" * 70)

print("\nAdvanced SQL techniques demonstrated:")
print("  1. Window functions (running totals, rankings)")
print("  2. Common Table Expressions (multi-step logic)")
print("  3. Date manipulation (cohort analysis)")
print("  4. Percentage calculations (distribution analysis)")
print("  5. Multiple simultaneous rankings")

print("\nQueries written:")
print("  - Query 11: Running total revenue")
print("  - Query 12: Customer LTV segmentation")
print("  - Query 13: Customer acquisition cohorts")
print("  - Query 14: Basket size analysis")
print("  - Query 15: State performance rankings")

print("\nFiles created:")
print("  - notebooks/05_advanced_sql.ipynb")
print("  - sql/advanced_queries.sql")

print("\n" + "=" * 70)
print("KEY FINDINGS SUMMARY")
print("=" * 70)

# Finding 1: Revenue growth
final_running_total = result11.iloc[-1]['running_total_revenue']
print(f"\n1. REVENUE TRAJECTORY")
print(f"   Total cumulative revenue: R$ {final_running_total:,.2f}")
print(f"   Peak growth month: {peak_growth_month['order_year']}-{peak_growth_month['order_month']:02.0f} ({peak_growth_month['mom_growth_pct']:.1f}% MoM growth)")
print(f"   Recent trend: {result11['mom_growth_pct'].tail(3).mean():.1f}% average growth (last 3 months)")

# Finding 2: Customer value concentration
print(f"\n2. CUSTOMER VALUE CONCENTRATION")
print(f"   High-value customers: {high_value['pct_of_customers']:.2f}% of base, {high_value['pct_of_revenue']:.2f}% of revenue")
print(f"   Average high-value LTV: R$ {high_value['avg_ltv']:,.2f}")
print(f"   Value multiplier: {ltv_multiplier:.1f}x (high vs low)")

# Finding 3: Acquisition efficiency
total_acquired = result13['customers_acquired'].sum()
print(f"\n3. CUSTOMER ACQUISITION")
print(f"   Total customers acquired: {total_acquired:,.0f}")
print(f"   Average monthly acquisition: {avg_monthly_acquisition:.0f} customers")
print(f"   Peak month: {peak_acquisition_month['cohort_month']} ({peak_acquisition_month['customers_acquired']:.0f} customers)")

# Finding 4: Cross-sell opportunity
print(f"\n4. CROSS-SELL OPPORTUNITY")
print(f"   Single-item orders: {single_item_pct:.1f}% of total")
print(f"   AOV uplift potential: {aov_uplift:.1f}% (if single-item → multi-item)")
print(f"   Revenue opportunity: R$ {(single_item_aov * aov_uplift / 100) * (result14[result14['basket_size'] == '1 item']['order_count'].values[0]):,.2f}")

# Finding 5: Regional optimization
print(f"\n5. REGIONAL PERFORMANCE")
print(f"   States analyzed: {len(result15)}")
print(f"   Largest performance gap: {top_gap_states.iloc[0]['customer_state']} (gap of {top_gap_states.iloc[0]['rank_gap']:.0f} ranks)")
print(f"   States needing attention: {len(result15[result15['rank_gap'] > 10])}")

print("\n" + "=" * 70)
print("STRATEGIC RECOMMENDATIONS")
print("=" * 70)

print("\n1. REVENUE GROWTH STRATEGY")
print("   Action: Replicate success factors from peak growth month")
print("   Expected impact: +15-20% MoM growth in targeted months")

print("\n2. CUSTOMER RETENTION FOCUS")
print("   Action: Move 10% of low-value to medium-value through engagement")
print("   Expected impact: +R$ 500K annual revenue")

print("\n3. ACQUISITION OPTIMIZATION")
print("   Action: Invest in channels driving peak acquisition months")
print("   Expected impact: +20% monthly acquisition rate")

print("\n4. CROSS-SELL PROGRAM")
print(f"   Action: Bundle recommendations for single-item orders")
print(f"   Expected impact: +{aov_uplift:.0f}% AOV on 30% of single-item orders")
print(f"   Revenue potential: R$ {(single_item_aov * aov_uplift / 100) * (result14[result14['basket_size'] == '1 item']['order_count'].values[0]) * 0.30:,.2f}")

print("\n5. REGIONAL RESOURCE ALLOCATION")
print("   Action: Fix logistics in high-revenue, poor-delivery states")
print("   Expected impact: 10-15% delivery time reduction")

print("\n" + "=" * 70)
print("NEXT STEPS: DAY 7")
print("=" * 70)
print("  - Create comprehensive README documentation")
print("  - Prepare visualization data for Streamlit (Week 2)")
print("  - Export query results to CSV for dashboarding")
print("  - Update LinkedIn profile with project progress")
print("  - Git commit: 'Day 6 complete - Advanced SQL mastery'")

# Close database connection
conn.close()
print("\nDatabase connection closed")
print("=" * 70)


BUSINESS INSIGHTS ACROSS ALL ADVANCED QUERIES

----------------------------------------------------------------------
INSIGHT 1: REVENUE GROWTH TRAJECTORY
----------------------------------------------------------------------

Month-over-month growth rates:
 order_year  order_month  monthly_revenue  mom_growth_pct
       2017            9        701169.99            8.54
       2017           10        751140.27            7.13
       2017           11       1153528.05           53.57
       2017           12        843199.17          -26.90
       2018            1       1078606.86           27.92
       2018            2        966510.88          -10.39
       2018            3       1120678.00           15.95
       2018            4       1132933.95            1.09
       2018            5       1128836.69           -0.36
       2018            6       1012090.68          -10.34
       2018            7       1027903.86            1.56
       2018            8        985414.28    